In [ ]:
# 导入必要的包以及定义函数
import os
import glob
import base64
import time
import requests
import json
import trimesh
import urllib
import numpy as np

def _create_colors():
    # 20 high contrast colors
    colors = [[230, 25, 75,255],[60, 180, 75,255],[255, 225, 25,255],\
            [0, 130, 200,255],[245, 130, 48,255],[145, 30, 180,255],[70, 240, 240,255],\
            [240, 50, 230,255],[210, 245, 60,255],[250, 190, 190,255],[0, 128, 128,255],\
            [230, 190, 255,255],[170, 110, 40,255],[255, 250, 200,255],[128, 0, 0,255],\
            [170, 255, 195,255],[128, 128, 0, 255]]
    #np.random.shuffle(colors)
    # gum color
    colors = [[255,255,255,255]] + colors
    return colors

def colored_mesh(mesh, label):
    COLORS = _create_colors()
    mcopy = mesh.copy()
    for i, l in enumerate(np.unique(label)):
        mcopy.visual.face_colors[np.where(label == l)[0]] = COLORS[i % 18]
    return mcopy

# 定义调用规则

请根据您从我方获取的信息修改以下代码块

In [ ]:
# 朝厚服务请求地址，随api文档发送
base_url = "<服务请求地址>"

# 朝厚文件服务地址，随api文档发送
file_server_url = "<服务文件服务器地址>"

# 必须传入鉴权 Header。请保护好TOKEN!!! 如果泄露请立即联系我们重置，所有使用该TOKEN的任务都会向您的账户计费
zh_token = "<贵司服务Token, 随合同发送>" # 调用所有的API都必须传入token用作鉴权

user_group = "APIClient" # 用户组，一般为 APIClient

# 贵司user_id, 随api文档发送
user_id = "<贵司user_id>"

# 如果您收到了creds.json, 下面将直接读取
if os.path.exists('creds.json'):
    creds = json.load(open('creds.json', 'r'))
    base_url = creds['base_url']
    file_server_url = creds['file_server_url']
    zh_token = creds['zh_token']
    user_id = creds['user_id']
    print("loaded creds from creds.json")

In [ ]:
# 以下请求数据以分牙+牙轴任务举例，依据您与我方签订的合约，您可能没有权限调用该样例API。

# 定义调用JSON的基础部分 - 调用的服务名称以及您的用户名信息
json_call = {
  "spec_group": "mesh-processing", # 调用的工作流组， 随API文档发送
  "spec_name": "oral-seg-and-axis", # 调用的工作流名称， 随API文档发送, 
  "spec_version": "1.0-snapshot", # 调用的工作流版本，随API文档发送
  "user_group": user_group,
  "user_id": user_id
}

### 定义回调地址

回调会在工作流运行结束后向指定地址发送一个POST请求，请求内容为一个JSON，内含4个字段

1. workflow_id (str): 对应的workflow_id
2. metadata (dict): 您启动工作流时传入的metadata
3. success (bool): 工作流是否成功（true为成功）
4. reason (str or null): 如果success为true, 该项为null. 否则为失败原因的字符串表示

如果需要回调，请uncomment以下代码块

In [ ]:
# json_call['metadata'] = { #（可选）这里可以加入metadata, 回调时会附带该信息，字典每项限制128个字符
#     "case_id": "CH-123",
#     "case_name": "ABCDE"
# }
# json_call['notification'] =[ #（可选）这里可以加入回调地址
#     {"url": "https://www.baidu.com"} # 这里可以加入多个回调地址，每个都是{"url": "xxx"}格式
# ]

## 定义输入/输出块

输入块由JSON文件中的`input_data`定义。对于三维网格，我们支持以下**输入**文件格式

```
"obj"
"stl"
"off"
"ply"
"glb"
"zip": zip内必须有且仅有一个mesh文件，且其后缀名必须为 obj, stl, off, ply 或 glb
"tar.gz": tar 或者 tar.gz内必须有且仅有一个mesh文件，且其后缀名必须为 obj, stl, off, ply 或 glb
"drc"
```

和以下**输出**文件格式
```
"obj"
"stl"
"off"
"ply"
"glb"
"drc"
```


我们**强烈推荐**在输入/输出中使用`drc`文件格式以减少网络传输时间，生成和读取drc格式文件可以使用Google的Draco库: https://github.com/google/draco

下表显示了同一个三维网格在不同文件格式下的文件大小 (点精度： 0.001mm）

| 文件格式 | 文件大小 |
| --- | --- |
| stl | 16.7M |
| obj | 13.5M |
| off | 14.2M |
| ply | 7.0M |
| glb | 6.0M |
| drc | 517.3K |


对于三位网格，输入可以采用两种形式传入：

1. 先将文件传入我们的文件服务系统，再将文件指针写入JSON调用串并调用API
2. 直接在API的JSON调用串中传入base64编码后的二进制数据

对于三位网格，输出可以指定两种形式传出：

1. 在API数据接口中返回文件指针，用户去文件系统下载具体二进制文件
2. 直接在API的数据接口中将二进制数据以base64编码形式返回


我们**强烈推荐**使用第一种方式，理由如下：

1. base64编码将增大数据流大小，提高网络延迟
2. 为保障接口性能，朝厚将拒绝过大的API请求。因此对于大文件，第二种方式会调用失败（HTTP CODE 413)

### 使用文件方式上传

In [ ]:
# 如果使用base64方式上传，请不要调用这个代码块
data = open('l.stl', 'rb').read()
resp = requests.get(file_server_url + f"/scratch/{user_group}/{user_id}/upload_url?" +
                    "postfix=stl", # 必须指定 postfix, 即文件后缀名
                    headers={"X-ZH-TOKEN": zh_token}) # 获取带签名的上传地址
resp.raise_for_status()

upload_url = resp.text[1:-1] # 返回为一个单字符串JSON "string", 这里也可以用json.loads(resp.text)

resp = requests.put(upload_url, data) # 上传至云储存服务不需要带鉴权头

resp.raise_for_status()
path = "/".join(urllib.parse.urlparse(upload_url).path.lstrip("/").split("/")[3:])
urn = f"urn:zhfile:o:s:{user_group}:{user_id}:{path}"
print("文件指针:", urn)

json_call["input_data"] = { # 输入数据，此处依照不同的任务字典内容会有不同，具体参照API文档，该处以分牙+牙轴举例
    "mesh": {"type":"stl", "data": urn}, # 文件指针
    "jaw_type": 'Lower' # 上颌是Upper, 下颌是Lower
}

# 定义输出配置（输出与输入不一定需要一样，比如可以输入文件stl, 输出base64的drc
json_call['output_config'] = {
    "mesh": {"type": "ply"} # 如果没有output-location字段，默认返回文件指针
}

### 使用base64方式上传

如果需要使用，请uncomment以下代码块

In [ ]:
# json_call["input_data"] = { # 输入数据，此处依照不同的任务字典内容会有不同，具体参照API文档，该处以分牙+牙轴举例
#     "mesh": {"type":"stl", "data": base64.b64encode(open('l.stl', 'rb').read()).decode()}, # 文件base64编码
#     "jaw_type": 'Lower' # 上颌是Upper, 下颌是Lower
# }

# # 定义输出配置（输出与输入不一定需要一样，比如可以输入文件stl, 输出base64的drc
# json_call['output_config'] = {
#     "mesh": {"type": "ply", "output-location": "bytes"} #下载也使用base64直接从接口获取
# }

# 提交请求

使用 `/run` POST方法提交请求

In [ ]:
headers = {
  "Content-Type": "application/json",
  "X-ZH-TOKEN": zh_token
}

url = base_url + '/run'

response = requests.request("POST", url, headers=headers, data=json.dumps(json_call))
response.raise_for_status()
create_result = response.json()
run_id = create_result['run_id']

# 打印任务号
print(run_id)

# 等待请求完成

使用轮询API等待请求完成，API地址为 `/run/{run_id}`

**强烈推荐使用回调方法**, 参考[定义回调地址](#定义回调地址)，回调方法无论工作流成功与否，在结束时均会向给定地址发送回调信息，您无需轮询结果

In [ ]:
# 第二步： 轮询状态
url = base_url + f"/run/{run_id}"

start_time = time.time()
while time.time()-start_time < 180: # 最多等3分钟
    time.sleep(3) # 轮询间隔
    response = requests.request("GET", url, headers=headers)
    result = response.json()
    if result['completed'] or result['failed']:
        break
    
if not result['completed']:
    if result['failed']:
        raise ValueError("API运行错误，原因： " + str(result['reason_public']))
    raise TimeoutError("API超时")

print("API运行时间： {}s".format(time.time()-start_time))

# 获取请求结果

使用 `/data/{run_id}` 获得该工作流所有输出数据，使用 `/data/{run_id}/{key}` 获取某一项数据

In [ ]:
url = base_url + f"/data/{run_id}"
response = requests.request("GET", url, headers=headers)
result = response.json()

In [ ]:
if result['mesh']['data'][:3] == "urn": # 使用urn方式返回， 需要去文件系统中拉取
    resp = requests.get(file_server_url + f"/file/download?" + urllib.parse.urlencode({
                        "urn": result['mesh']['data']}),
                        headers={"X-ZH-TOKEN": zh_token})
    mesh_bytes = resp.content
else: # 使用bytes方式返回，直接读取
    mesh_bytes = base64.b64decode(result['mesh']['data'])
result_mesh = trimesh.load(trimesh.util.wrap_as_stream(mesh_bytes), file_type=result['mesh']['type'])

# 样例API输出结果展示

以下为该分牙+牙轴样例API的输出结果展示。该部分之上代码对所有API通用，以下部分仅针对分牙+牙轴输出处理及展示

In [ ]:
print('牙号列表：', np.unique(result['seg_labels']))
colored_mesh(result_mesh, result['seg_labels']).show()

In [ ]:
# 将所有牙号转到局部坐标系
def _get_vid_new_faces_by_fid(mesh, inds):
    old_faces = mesh.faces[inds].reshape(-1)
    vid = np.unique(old_faces)
    vid_map = dict(zip(vid, np.arange(len(vid))))
    faces = np.array([vid_map[x] for x in old_faces]).reshape(-1, 3)
    return vid, faces

def get_mesh_by_id(mesh, inds):
    vid, faces = _get_vid_new_faces_by_fid(mesh, inds)
    return trimesh.Trimesh(mesh.vertices[vid], faces)
def split_tooth(mesh, label):
    splitted = {}
    unique_labels = np.unique(label)

    for fdi in unique_labels:
        if fdi == 0:
            continue
        idx = np.where(label == fdi)[0]
        splitted[str(int(fdi))] = get_mesh_by_id(mesh, idx)
    return splitted

all_t = None
for k, t in split_tooth(result_mesh, result['seg_labels']).items():
    all_t += t.apply_transform(result['axis'][k])

all_t.show()